# Tutorial 2: IntroducEinführung in PyTorch

Willkommen zu unserem PyTorch-Tutorial für den Deep-Learning-Kurs 2023 an der Universität Amsterdam! Dieses Notebook bietet eine kurze Einführung in die Grundlagen von PyTorch und bereitet dich darauf vor, eigene neuronale Netze zu erstellen. PyTorch ist ein Open-Source-Framework für maschinelles Lernen, das dir flexibles Erstellen und effizientes Optimieren neuronaler Netze ermöglicht. PyTorch ist allerdings nicht das einzige Framework dieser Art. Alternativen sind [TensorFlow](https://www.tensorflow.org/), [JAX](https://github.com/google/jax#quickstart-colab-in-the-cloud) und [Caffe](http://caffe.berkeleyvision.org/). Wir haben uns an der Universität Amsterdam für PyTorch entschieden, da es gut etabliert ist, eine große Entwickler-Community hat (ursprünglich von Facebook entwickelt), sehr flexibel ist und insbesondere in der Forschung eingesetzt wird. Viele aktuelle wissenschaftliche Veröffentlichungen stellen ihren Code in PyTorch bereit, daher ist es von Vorteil, damit vertraut zu sein. TensorFlow (entwickelt von Google) ist hingegen primär als produktionsreife Deep-Learning-Bibliothek bekannt. Wenn du ein Machine-Learning-Framework bereits gut beherrschst, ist es leicht, ein anderes zu erlernen, da viele auf den gleichen Konzepten und Ideen basieren. Zum Beispiel wurde TensorFlow in Version 2 stark von den beliebtesten Features von PyTorch inspiriert, was die Frameworks einander noch ähnlicher gemacht hat. Falls du bereits mit PyTorch vertraut bist und eigene neuronale Netzwerk-Projekte erstellt hast, kannst du dieses Notebook gerne nur überfliegen.

Natürlich sind wir nicht die Ersten, die ein PyTorch-Tutorial erstellen. Es gibt viele großartige Tutorials im Internet, darunter der ["60-min blitz"](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html) auf der offiziellen [PyTorch-Webseite](https://pytorch.org/tutorials/). Dennoch haben wir uns entschieden, ein eigenes Tutorial zu gestalten. Dieses soll dir die Grundlagen vermitteln, die du speziell für unsere praktischen Übungen benötigst, und dir trotzdem ein Verständnis der Funktionsweise von PyTorch geben. Im Laufe der nächsten Wochen werden wir auch weiterhin neue PyTorch-Features in dieser Reihe von Jupyter-Notebook-Tutorials rund um Deep Learning erkunden.

Wir werden einen Satz von Standard-Bibliotheken verwenden, die häufig in Machine-Learning-Projekten genutzt werden. Wenn du dieses Notebook auf Google Colab ausführst, sollten alle Bibliotheken vorinstalliert sein. Falls du das Notebook lokal ausführst, stelle sicher, dass du unsere `d12023`-Umgebung installiert und aktiviert hast.

In [1]:
## Standard libraries
import os
import math
import numpy as np
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()

## Progress bar
from tqdm.notebook import tqdm

/var/folders/nj/yxgl90t15y58nn503xvrtr4w0000gn/T/ipykernel_87459/47578708.py:11: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


## Die Grundlagen von PyTorch

Wir beginnen mit einem Überblick über die grundlegenden Konzepte von PyTorch. Als Voraussetzung empfehlen wir, mit dem `numpy`-Paket vertraut zu sein, da die meisten Machine-Learning-Frameworks auf sehr ähnlichen Konzepten basieren. Falls du numpy noch nicht kennst, keine Sorge: hier findest du ein [Tutorial](https://numpy.org/devdocs/user/quickstart.html), um dich einzuarbeiten.

Los geht's! Beginnen wir damit, PyTorch zu importieren. Das Paket heißt `torch`, angelehnt an das ursprüngliche Framework Torch. Als ersten Schritt können wir die Version überprüfen:

In [2]:
import torch
print("Using torch", torch.__version__)

Using torch 2.2.1


Zum Zeitpunkt der Erstellung dieses Tutorials (Ende Oktober 2023) ist die aktuelle stabile Version 2.1. Du solltest also als Ausgabe `Using torch 2.1.0` oder `Using torch 2.0.0` sehen, möglicherweise ergänzt durch die Angabe der CUDAVersion auf Colab. Falls du die `d12023`-Umgebung verwendest, solltest du `Using torch 2.1.0` sehen. Generell empfehlen wir, die PyTorch-Version stets auf dem aktuellsten Stand zu halten. Solltest du eine Versionsnummer unter 2.0 sehen, stelle sicher, dass du die richtige Umgebung installiert hast oder frage deine TAs. Im Fall, dass PyTorch 2.2 oder neuer während des Kurses veröffentlicht wird, brauchst du dir keine Sorgen zu machen. Die Schnittstelle zwischen PyTorch-Versionen ändert sich nicht drastisch, daher sollte der Code auch mit neueren Versionen lauffähig sein.

Wie in jedem Machine-Learning-Framework bietet PyTorch Funktionen für zufällige Vorgänge, wie die Erzeugung von Zufallszahlen. Es ist jedoch empfehlenswert, deinen Code so zu gestalten, dass Ergebnisse mit denselben Zufallszahlen reproduzierbar sind. Dies ermöglicht es dir, Fehler aufzuspüren und sicherzustellen, dass Änderungen deinerseits (und nicht zufällige Fluktuationen) die Ursache für veränderte Ergebnisse sind. Deshalb setzen wir nachfolgend einen "Seed" (Startwert). Der Seed ist der Ausgangspunkt für den Zufallszahlengenerator. Durch Setzen eines festen Seeds stellst du sicher, dass die Sequenz der erzeugten Zufallszahlen immer gleich ist.

In [3]:
torch.manual_seed(42) # hier setzen wir den seed

### Tensoren

Tensoren sind das PyTorch-Äquivalent zu NumPy-Arrays, bieten aber zusätzlich Unterstützung für GPU-Beschleunigung (dazu später mehr). Der Name "Tensor" ist eine Verallgemeinerung von Konzepten, die du bereits kennst. Ein Vektor ist zum Beispiel ein 1D-Tensor und eine Matrix ein 2DTensor. Bei der Arbeit mit neuronalen Netzen verwenden wir Tensoren unterschiedlicher Formen und Dimensionen.

Die meisten gängigen Funktionen, die du von NumPy kennst, können auch auf Tensoren angewendet werden. Da NumPy-Arrays und Tensoren sich stark ähneln, können wir die meisten Tensoren in NumPy-Arrays (und umgekehrt) umwandeln. Dies ist jedoch in der Regel nicht oft notwendig.

#### Initialisierung

Fangen wir damit an, uns verschiedene Möglichkeiten zur Erstellung eines Tensors anzusehen. Es gibt viele Optionen, die einfachste ist der Aufruf von `torch.Tensor` mit der gewünschten Form als Eingabeparameter:

In [4]:
x = torch.Tensor(2, 3, 4)
print(x)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])


Die Funktion `torch.Tensor` weist den gewünschten Speicher für den Tensor zu, verwendet aber bereits vorhandene Speicherinhalte ggf. weiter. Um dem Tensor direkt bei der Initialisierung Werte zuzuweisen, gibt es verschiedene Möglichkeiten, darunter:
- `torch.zeros`: Erzeugt einen Tensor gefüllt mit Nullen.
- `torch.ones`: Erzeugt einen Tensor gefüllt mit Einsen.
- `torch.rand`: Erzeugt einen Tensor mit Zufallswerten, die gleichmäßig zwischen 0 und 1 verteilt sind.
- `torch.randn`: Erzeugt einen Tensor mit Zufallswerten, die einer Normalverteilung mit Mittelwert 0 und Varianz 1 folgen.
- `torch.arange`: Erzeugt einen Tensor mit den Werten $N, N+1, N+2, \ldots, M$
- `torch.Tensor` (input list): Erzeugt einen Tensor aus einer von dir bereitgestellten Liste.

In [5]:
# Tensor aus (verschachtelter) Liste erzeugen
x = torch.Tensor([[1, 2], [3, 4]])
print(x)

tensor([[1., 2.],
        [3., 4.]])


In [6]:
# Erstelle einen Tensor mit Zufallswerten zwischen 0 und 1 mit der Form [2, 3, 4]
x = torch.rand(2, 3, 4)
print(x)

tensor([[[0.8823, 0.9150, 0.3829, 0.9593],
         [0.3904, 0.6009, 0.2566, 0.7936],
         [0.9408, 0.1332, 0.9346, 0.5936]],

        [[0.8694, 0.5677, 0.7411, 0.4294],
         [0.8854, 0.5739, 0.2666, 0.6274],
         [0.2696, 0.4414, 0.2969, 0.8317]]])


Du kannst die Form (Shape) eines Tensors wie in NumPy ermitteln (`x.shape`) oder alternativ mit der `.size`-Methode

In [7]:
shape = x.shape
print("Shape", x.shape)

size = x.size()
print("Size", x.size())

dim1, dim2, dim3 = x.size()
print("Size", dim1, dim2, dim3)

Shape torch.Size([2, 3, 4])
Size torch.Size([2, 3, 4])
Size 2 3 4


#### Konvertierung: Tensor ↔ NumPy

Tensoren können in NumPy-Arrays umgewandelt werden und umgekehrt. Um ein NumPy-Array in einen Tensor zu verwandeln, verwenden wir die Funktion `torch.from_numpy`:

In [8]:
np_arr = np.array([[1, 2], [3, 4]])
tensor = torch.from_numpy(np_arr)

print("Numpy array", np_arr)
print("Tensor", tensor)

Numpy array [[1 2]
 [3 4]]
Tensor tensor([[1, 2],
        [3, 4]])


Um einen PyTorch Tensor in ein NumPy-Array zurückzuverwandeln, nutzen wir die `.numpy()`-Methode des Tensors:

In [9]:
tensor = torch.arange(4)
np_arr = tensor.numpy()

print("Tensor", tensor)
print("Numpy array", np_arr)

Tensor tensor([0, 1, 2, 3])
Numpy array [0 1 2 3]


Die Umwandlung von Tensoren in NumPy-Arrays setzt voraus, dass sich der Tensor auf der CPU (dem Hauptprozessor) und nicht auf der GPU (Grafikprozessor) befindet. (Mehr zur GPU-Unterstützung später.) Falls dein Tensor auf der GPU ist, musst du zuerst die  `.cpu()`-Methode darauf anwenden. Das sieht dann so aus: `np_arr = tensor.cpu().numpy()`.

#### Operationen

Die meisten Operationen, die in NumPy verfügbar sind, existieren auch in PyTorch. Eine vollständige Liste findest du in der [PyTorch-Dokumentation](https://pytorch.org/docs/stable/tensors.html#). Wir werden die wichtigsten hier besprechen.

Die grundlegendste Operation - Zwei Tensoren elementweise addieren:

In [10]:
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
y = x1 + x2

print("X1", x1)
print("X2", x2)
print("Y", y)

X1 tensor([[0.1053, 0.2695, 0.3588],
        [0.1994, 0.5472, 0.0062]])
X2 tensor([[0.9516, 0.0753, 0.8860],
        [0.5832, 0.3376, 0.8090]])
Y tensor([[1.0569, 0.3448, 1.2448],
        [0.7826, 0.8848, 0.8151]])


Die Aufrufe `x1 + x2` erzeugt einen neuen Tensor, der die Summe der beiden Eingaben enthält. Wir können jedoch auch In-Place-Operationen verwenden, die direkt auf den Speicher eines Tensors wirken. Dadurch verändern wir den Inhalt von `x2` ohne die Möglichkeit, auf Werte vor der Operation zuzugreifen. Ein Beispiel dafür siehst du unten:

In [11]:
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
print("X1 (before)", x1)
print("X2 (before)", x2)

x2.add_(x1)
print("X1 (after)", x1)
print("X2 (after)", x2)

X1 (before) tensor([[0.5779, 0.9040, 0.5547],
        [0.3423, 0.6343, 0.3644]])
X2 (before) tensor([[0.7104, 0.9464, 0.7890],
        [0.2814, 0.7886, 0.5895]])
X1 (after) tensor([[0.5779, 0.9040, 0.5547],
        [0.3423, 0.6343, 0.3644]])
X2 (after) tensor([[1.2884, 1.8504, 1.3437],
        [0.6237, 1.4230, 0.9539]])


In-Place-Operationen werden üblicherweise durch ein angehängtes Unterstrich-Zeichen gekennzeichnet (z.B. "`add_`" anstelle von "`add`").

Eine weitere häufige Operation ist das Ändern der Form eines Tensors. Ein Tensor der Größe (2,3) kann in jede andere Form mit derselben Anzahl von Elementen umgewandelt werden (z.B. ein Tensor der Größe (6), oder (3,2) usw.). In PyTorch heißt diese Operation `view`:

In [12]:
x = torch.arange(6)
print("X", x)

X tensor([0, 1, 2, 3, 4, 5])


In [13]:
x = x.view(2, 3)
print("X", x)

X tensor([[0, 1, 2],
        [3, 4, 5]])


In [14]:
x = x.permute(1, 0) # Vertauschen der Dimensionen 0 und 1
print("X", x)

X tensor([[0, 3],
        [1, 4],
        [2, 5]])


Weitere häufig verwendete Operationen sind Matrixmultiplikationen, die für neuronale Netze unerlässlich sind. Oft haben wir einen Eingabevektor $\mathbf{x}$, der mithilfe einer gelernten Gewichtsmatrix $\mathbf{W}$ transformiert wird. Es gibt verschiedene Wege und Funktionen, um Matrixmultiplikationen durchzuführen. Einige davon sind unten aufgeführt:
- `torch.matmul`: Führt eine Matrixmultiplikation zweier Tensoren durch. Das spezifische Verhalten hängt von der Dimensionalität ab. Sind beide Tensoren 2D (Matrizen), wird das klassische Matrixprodukt berechnet. Bei höherdimensionalen Tensoren wird Broadcasting unterstützt (Details siehe [Dokumentation](https://pytorch.org/docs/stable/generated/torch.matmul.html?highlight=matmul#torch.matmul)). Kann alternativ mit dem Operator `@` verwendet werden, ähnlich wie in NumPy.
- `torch.mm`: Führt eine Matrixmultiplikation zweier Matrizen durch. Broadcasting wird nicht unterstützt (siehe [Dokumentation](https://pytorch.org/docs/stable/generated/torch.mm.html?highlight=torch%20mm#torch.mm)).
- `torch.bmm`: Führt eine Matrixmultiplikation unter Berücksichtigung einer Batch-Dimension durch. Hat der erste Tensor $T$ die Form ($b \times n \times m$) und der zweite Tensor $R$ die Form ($b \times m \times p$), dann hat die Ausgabe $O$ die Form ($b \times n \times p$). Dieser Output entsteht durch $b$ separate Matrixmultiplikationen jeweils entsprechender Teilmatrizen von $T$ und $R$: $O_i=T_i @ R_i$
- `torch.einsum`: Führt Matrixmultiplikationen und andere Berechnungen (z. B. Summen von Produkten) mithilfe der Einsteinschen Summenkonvention durch. Eine Erklärung der Einsteinschen Summenkonvention findest du in Übung 1.


Normalerweise verwenden wir `torch.matmul` oder `torch.bmm`. Im Folgenden können wir eine Matrixmultiplikation mit `torch.matmul` ausprobieren.

In [15]:
x = torch.arange(6)
x = x.view(2, 3)
print("X", x)

X tensor([[0, 1, 2],
        [3, 4, 5]])


In [16]:
W = torch.arange(9).view(3, 3) # Wir können mehrere Operationen sequentiell anwenden, indem wir sie in einer einzigen Zeile zusammenfassen.
print("W", W)

W tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])


In [17]:
h = torch.matmul(x, W)
print("h", h)

h tensor([[15, 18, 21],
        [42, 54, 66]])


#### Indexing

Wir müssen häufig einen bestimmten Teil eines Tensors auswählen. Die Indizierung funktioniert genau wie in NumPy. Probieren wir es aus:

In [18]:
x = torch.arange(12).view(3, 4)
print("X", x)

X tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


In [19]:
print(x[:, 1]) # zweite spalte

tensor([1, 5, 9])


In [20]:
print(x[0]) # erste zeile

tensor([0, 1, 2, 3])


In [21]:
print(x[:2, -1]) # ersten beiden zeilen, davon die letzte spalte

tensor([3, 7])


In [22]:
print(x[1:3, :]) # ersten beiden zeilen

tensor([[ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


### Dynamische Berechnungsgraphen und Backpropagation

Einer der Hauptgründe, PyTorch in Deep-Learning-Projekten einzusetzen, ist die Möglichkeit, **Gradienten (Ableitungen)** von definierten Funktionen automatisch zu berechnen. Da neuronale Netze im Wesentlichen komplexe Funktionen sind, deren Gewichtsmatrizen wir "lernen" möchten, benötigen wir diese Funktionalität.  Gewichtsmatrizen, die wir optimieren wollen, werden auch als **Parameter** oder schlicht **Gewichte** bezeichnet.

Falls unser neuronales Netz nur einen einzelnen Ausgabewert erzeugen würde, würden wir schlicht von der **Ableitung** sprechen. Da wir jedoch häufig **mehrere** Ausgabevariablen ("Werte") haben werden, verwenden wir den Begriff **Gradienten**. Er ist der allgemeinere Begriff.

Gegeben eine Eingabe $x$, definieren wir unsere Funktion, indem wir diese Eingabe **manipulieren**. Üblicherweise geschieht das durch Matrixmultiplikationen mit Gewichtsmatrizen und Additionen mit sogenannten Bias-Vektoren. Während dieser Manipulationen erstellt PyTorch automatisch einen **Berechnungsgraphen**. Dieser zeigt, wie wir von der Eingabe zur Ausgabe gelangen. PyTorch folgt dem "**Define-by-Run**"-Prinzip: Wir führen die Berechnungen einfach aus und PyTorch hält den Graphen für uns fest. So entsteht dynamisch (Schritt für Schritt) ein Berechnungsgraph.

Zusammengefasst:  Wir müssen lediglich die **Ausgabe** berechnen und können dann PyTorch anweisen, automatisch die **Gradienten** zu bestimmen.

**Hinweis: Wozu brauchen wir Gradienten?**  Angenommen, wir haben eine Funktion (ein neuronales Netz) definiert, die für eine Eingabe $\mathbf{x}$ eine bestimmte Ausgabe $y$ berechnen soll. Dazu definieren wir zusätzlich eine **Fehlerfunktion**, die angibt, wie "falsch" unser Netz liegt – wie schlecht es darin ist, $y$ aus $\mathbf{x}$ vorherzusagen. Mithilfe der Gradienten können wir nun die Gewichte $\mathbf{W}$, die für die Ausgabe verantwortlich waren, **aktualisieren**. Bei der nächsten Eingabe von $\mathbf{x}$ wird die Ausgabe dann näher an unserem gewünschten Ergebnis liegen.


Als erstes müssen wir festlegen, für welche Tensoren Gradienten berechnet werden sollen. Standardmäßig werden Tensoren ohne die Gradienten-Berechnung erstellt. Dies geschieht aus Performance-Gründen, da die Berechnung und Speicherung der Gradienten aufwändig sein kann.

In [23]:
x = torch.ones((3,))
print(x.requires_grad)

False


Wir können dies für einen bestehenden Tensor ändern, indem wir die Funktion `requires_grad_()` verwenden (der Unterstrich zeigt an, dass es sich um eine In-Place-Operation handelt). Alternativ kannst du bei der Erstellung eines Tensors das Argument `requires_grad=True` an die meisten Initialisierer übergeben, die wir bisher kennengelernt haben.

In [24]:
x.requires_grad_(True)
print(x.requires_grad)

True


Um uns mit dem Konzept des Berechnungsgraphen vertraut zu machen, werden wir einen für die folgende Funktion erstellen:  

$$
y=\frac{1}{\ell(x)} \sum_i\left[\left(x_i+2\right)^2+3\right]
$$  

dabei ist  $\ell(x)$  die Anzahl der Elemente in $x$. Kurz gesagt berechnen wir einen Mittelwert über den Ausdruck innerhalb der Summe. Nehmen wir an, $x$ enthält Parameter, und wir möchten die Ausgabe $y$ optimieren (entweder maximieren oder minimieren). Dazu müssen wir die Gradienten  $\partial y / \partial \mathbf{x}$  bestimmen. Als Beispiel verwenden wir die Eingabe $\mathbf{x}=[0,1,2]$.



In [25]:
x = torch.arange(3, dtype=torch.float32, requires_grad=True) # Nur Gleitkomma-Tensoren können Gradienten haben.
print("X", x)

X tensor([0., 1., 2.], requires_grad=True)


Lass uns den Berechnungsgraphen nun Schritt für Schritt aufbauen. Mehrere Operationen könnten zwar in einer Zeile kombiniert werden, aber wir werden sie hier einzeln ausführen. Dies hilft uns besser zu verstehen, wie jede Operation zum Berechnungsgraphen hinzugefügt wird.

In [26]:
a = x + 2
b = a ** 2
c = b + 3
y = c.mean()

print("Y", y)

Y tensor(12.6667, grad_fn=<MeanBackward0>)


Mit den obigen Anweisungen haben wir einen Berechnungsgraphen erstellt, der in etwa so aussieht wie in der folgenden Abbildung:

<img src="images/pytorch_computation_graph.svg" alt="Pytorch Computation Graph" />

Wir berechnen $a$ basierend auf den Eingaben $x$ und der Konstante $2$, danach wird $a$ quadriert und so weiter. Die Visualisierung abstrahiert die Abhängigkeiten zwischen Ein- und Ausgaben der verwendeten Operationen. Jeder Knoten im Berechnungsgraphen hat automatisch eine Funktion zur Berechnung der Gradienten bezüglich seiner Eingaben (`grad_fn`). Dies wurde deutlich, als wir den Ausgabetensor $y$ ausgegeben haben. Deshalb wird der Graph üblicherweise in umgekehrter Richtung dargestellt (Pfeile zeigen vom Ergebnis zu den Eingaben).  Wir können Backpropagation im Graphen durchführen, indem wir die `backward()`-Funktion auf der letzten Ausgabe aufrufen. So werden die Gradienten für jeden Tensor berechnet, der `requires_grad=True` erfüllt.

In [27]:
y.backward()

`x.grad` enthält nun den Gradienten  $\partial y / \partial x$. Dieser Gradient gibt an, wie eine Änderung in $\mathbf{x}$ die Ausgabe $\mathbf{y}$ beeinflusst (bei der aktuellen Eingabe $\mathbf{x}=[0,1,2]$):

In [28]:
print(x.grad)

tensor([1.3333, 2.0000, 2.6667])


Wir können die Gradienten auch manuell überprüfen. Dazu werden wir sie mithilfe der Kettenregel berechnen – genau wie PyTorch es getan hat:  

$$
\frac{\partial y}{\partial x_i}=\frac{\partial y}{\partial c_i} \frac{\partial c_i}{\partial b_i} \frac{\partial b_i}{\partial a_i} \frac{\partial a_i}{\partial x_i}
$$

Hinweis: Wir haben die Gleichung mit Indexnotation vereinfacht und nutzen dabei, dass  – abgesehen von der Mittelwertbildung  – keine Operation die Elemente des Tensors miteinander kombiniert. Die partiellen Ableitungen sind:  

$$
\frac{\partial a_i}{\partial x_i}=1, \quad \frac{\partial b_i}{\partial a_i}=2 \cdot a_i \quad \frac{\partial c_i}{\partial b_i}=1 \quad \frac{\partial y}{\partial c_i}=\frac{1}{3}
$$

Somit ergeben sich mit der Eingabe $\mathbf{x}=[0,1,2]$ die Gradienten $\partial y / \partial \mathbf{x}=[4 / 3,2,8 / 3]$. Die vorherige Codezelle sollte das gleiche Ergebnis ausgegeben haben.

### GPU-Unterstützung/GPU-Beschleunigung

Eine entscheidende Stärke von PyTorch ist die Unterstützung von GPUs (Grafikprozessoren). GPUs können viele tausend kleinere Operationen gleichzeitig ausführen. Das macht sie ideal für umfangreiche Matrixoperationen wie sie in neuronalen Netzen vorkommen. Im Vergleich von GPUs und CPUs lassen sich folgende Hauptunterschiede festhalten (Quelle: [Kevin Krewell, 2009](https://blogs.nvidia.com/blog/2009/12/16/whats-the-difference-between-a-cpu-and-a-gpu/)):

| **Merkmal**              | **CPU**                       | **GPU**                        |
| ------------------------ | ----------------------------- | ------------------------------ |
| Ausschreibung            | Zentrale Recheneinheit        | Grafikprozessor                |
| Anzahl der Kerne         | Einige Kerne                  | Viele Kerne                    |
| Latenz                   | Geringe Verzögerung           | Hoher Durchsatz                |
| Stärken                  | Gut für serielle Verarbeitung | Gut für parallele Verarbeitung |
| Operationen gleichzeitig | Eine Handvoll Operationen     | Tausende Operationen           |

CPUs und GPUs haben unterschiedliche Vor- und Nachteile. Deshalb sind in vielen Computern beide Komponenten verbaut, um sie für verschiedene Aufgaben zu nutzen. Falls du noch nicht mit GPUs vertraut bist, findest du weitere Informationen in diesem [NVIDIA Blogpost](https://blogs.nvidia.com/blog/2009/12/16/whats-the-difference-between-a-cpu-and-a-gpu/) oder [hier](https://www.intel.com/content/www/us/en/products/docs/processors/what-is-a-gpu.html).

GPUs können das Training deines neuronalen Netzes um den Faktor 100 (oder mehr) beschleunigen. Das ist essentiell für große Netzwerkarchitekturen. PyTorch bietet umfangreiche Funktionen zur Nutzung von GPUs (vor allem von NVIDIA, dank der Bibliotheken [CUDA](https://developer.nvidia.com/cuda-zone) und [cuDNN](https://developer.nvidia.com/cudnn)). Lass uns zunächst prüfen, ob eine GPU verfügbar ist:

In [29]:
gpu_avail = torch.cuda.is_available()
mps_avail = torch.backends.mps.is_available()
print(f"Ist eine GPU verfügbar? {gpu_avail}")
print(f"Ist ein MPS (Apple) verfügbar? {mps_avail}")

Ist eine GPU verfügbar? False
Ist ein MPS (Apple) verfügbar? True


Falls du eine GPU hast, obiger Befehl aber "`False`" zurückgibt, stelle sicher, dass die korrekte CUDA-Version installiert ist. Die `d12023`-Umgebung nutzt standardmäßig CUDA 11.8, passend für den Snellius-Supercomputer. Bitte ändere die Version falls nötig (CUDA 11.3 ist derzeit auf Colab verbreitet). Achte bei Google Colab darauf, dass du eine GPU in den Laufzeiteinstellungen ausgewählt hast (im Menü unter `Laufzeit -> Laufzeittyp ändern`).

Standardmäßig werden alle Tensoren, die du erstellst, auf der CPU gespeichert. Um einen Tensor auf die GPU zu übertragen, können wir die Funktion `.to(...)` oder `.cuda()` verwenden. Es empfiehlt sich jedoch, in deinem Code ein Geräteobjekt ("device object") zu definieren. Dieses verweist auf die GPU (sofern vorhanden), andernfalls auf die CPU. Wenn du deinen Code anschließend in Bezug auf dieses Geräteobjekt schreibst, kannst du ihn sowohl auf Systemen ohne GPU als auch auf solchen mit GPU bzw. auf Apple Geräten ab M1 Prozessor und neuer ausführen. (Hinweis: Ab PyTorch 1.12 ist die Verwendung von Apple MPS (Metal Performance Shaders) für die GPU-Beschleunigung auf Macs mit Apple Silicon möglich.)Probieren wir es aus. Die Optionen für `torch.device` finden sich in der [Dokumentation](https://pytorch.org/docs/stable/tensor_attributes.html#torch-device). So kannst du das Gerät festlegen:

In [30]:
# Überprüfe, ob Apple MPS verfügbar ist
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    # Falls MPS nicht verfügbar ist, verwende CUDA wenn verfügbar, sonst CPU
    torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
print("Device", device)

Device mps


Lass uns nun einen Tensor erstellen und ihn auf das zuvor festgelegte Gerät übertragen:

In [31]:
x = torch.zeros(2, 3)
x = x.to(device)

print("X", x)

X tensor([[0., 0., 0.],
        [0., 0., 0.]], device='mps:0')


Falls du eine GPU hast, solltest du nun das Attribut `device='cuda:0'` oder `device='mps:0'` neben deinem Tensor sehen. Die Null hinter cuda/mps  gibt an, dass dies die erste GPU in deinem System ist. PyTorch unterstützt auch Systeme mit mehreren GPUs. Dies wirst du jedoch erst für sehr große Netzwerke benötigen (bei Interesse findest du mehr in der [PyTorch-Dokumentation](https://pytorch.org/docs/stable/distributed.html#distributed-basics)). Wir können auch die Laufzeit einer großen Matrixmultiplikation auf der CPU und der GPU vergleichen:

In [32]:
x = torch.randn(5000, 5000)

## CPU Version
start_time = time.time()
_ = torch.matmul(x, x)
end_time = time.time()
print(f"Laufzeit auf der CPU: {(end_time - start_time):6.5f}s")

## GPU Version
if device.type == "mps":
    x = x.to(device)
    _ = torch.matmul(x, x)
    # MPS arbeitet asynchron. Daher müssen wir andere Funktionen zur Zeitmessung verwenden.
    start = torch.mps.Event(enable_timing=True)
    end = torch.mps.Event(enable_timing=True)
    start.record()
    _ = torch.matmul(x, x)
    end.record()
    torch.mps.synchronize()
    print(f"Laufzeit auf MPS: {0.001 * start.elapsed_time(end):6.5f}s")
elif device.type == "cuda":
    x = x.to(device)
    _ = torch.matmul(x, x)
    # CUDA arbeitet auch asynchron. Daher müssen wir andere Funktionen zur Zeitmessung verwenden.
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    _ = torch.matmul(x, x)
    end.record()
    torch.cuda.synchronize()
    print(f"Laufzeit auf CUDA: {0.001 * start.elapsed_time(end):6.5f}s")

Laufzeit auf der CPU: 0.32534s
Laufzeit auf MPS: 0.21464s


Abhängig von der Größe der Operation sowie deiner CPU/GPU, kann die Beschleunigung um mehr als das 50-fache betragen. Da Matrixmultiplikationen (`matmul`) in neuronalen Netzen sehr häufig sind, lässt sich der enorme Vorteil des GPU-basierten Trainings leicht erkennen. Die Laufzeitschätzung kann hier etwas ungenau sein, da wir sie nicht mehrfach durchgeführt haben. Du kannst das gerne erweitern, allerdings verlängert das dann auch die Ausführungszeit.

Bei der Generierung von Zufallszahlen sind die Startwerte ("Seeds") zwischen CPU und GPU nicht synchronisiert. Um reproduzierbare Ergebnisse zu gewährleisten, müssen wir den Seed daher separat auf der GPU setzen. Beachte, dass verschiedene GPU-Architekturen selbst bei identischem Code nicht zwangsläufig die gleichen Zufallszahlen erzeugen. Trotzdem möchten wir natürlich vermeiden, dass unser Code bei jeder Ausführung auf derselben Hardware unterschiedliche Ausgaben liefert. Deshalb setzen wir den Seed auch auf der GPU:

In [33]:
# Auch für GPU-Operationen muss ein separater Startwert (Seed) gesetzt werden.
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
elif torch.backends.mps.is_available():
    torch.mps.manual_seed(42)
    
# Zusätzlich sind manche Operationen auf der GPU aus Effizienzgründen stochastisch implementiert.
# Dies kann dazu führen, dass derselbe Code bei mehrfacher Ausführung leicht unterschiedliche Ergebnisse liefern kann.
# Stochastisch: Bedeutet in diesem Kontext, dass die Operation ein zufälliges Element enthält. Die Ausgabe ist also bei jedem Durchlauf nicht exakt gleich.
# Effizienz: Durch die stochastische Implementierung können bestimmte Operationen auf der GPU deutlich schneller ausgeführt werden, auch wenn das Ergebnis minimal variieren kann.
# Aus Gründen der Reproduzierbarkeit möchten wir sicherstellen, dass alle Operationen auf der GPU (falls verwendet) deterministisch ausgeführt werden.
if device.type == "cuda":
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## Lernen durch Beispiele: Kontinuierliches XOR

Wenn wir ein neuronales Netz in PyTorch implementieren möchten, könnten wir alle Parameter (Gewichtsmatrizen, Bias-Vektoren) als Tensoren definieren (`requires_grad=True`) und PyTorch die Gradienten berechnen lassen. Bei vielen Parametern wird das jedoch schnell unübersichtlich. Deshalb gibt es in PyTorch das Paket `torch.nn`, das die Erstellung neuronaler Netze deutlich vereinfacht.

Wir werden die Bibliotheken und alle zusätzlichen Komponenten vorstellen, die du für das Training eines neuronalen Netzes in PyTorch benötigst. Dazu nutzen wir einen einfachen Klassifikator an einem bekannten Beispiel: XOR. Gegeben seien zwei binäre Eingaben $x_1$ und $x_2$. Das vorherzusagende Label ist 1, wenn entweder $x_1$ oder $x_2$ gleich 1 ist (und das jeweils andere 0 ist), andernfalls ist das Label 0. Dieses Beispiel wurde berühmt, weil ein einzelnes Neuron (d.h. ein linearer Klassifikator) diese simple Funktion nicht lernen kann. Daher werden wir ein kleines neuronales Netz erstellen, das diese Aufgabe bewältigen kann. Um es etwas interessanter zu machen, verschieben wir das XOR-Problem in einen kontinuierlichen Raum und fügen den binären Eingaben etwas Gaußsches Rauschen hinzu. Eine mögliche Trennung in einem XOR-Datensatz könnte dann so aussehen:

<img src="images/continuous_xor.svg" alt="Continious XOR" />

### Das Modell

Das Paket `torch.nn` bietet eine Reihe nützlicher Klassen, wie lineare Netzwerk-Layer, Aktivierungsfunktionen, Verlustfunktionen usw. Eine vollständige Liste findest du [hier](https://pytorch.org/docs/stable/nn.html). Falls du einen bestimmten Netzwerk-Layer benötigst, lohnt es sich, zuerst in der Dokumentation des Pakets nachzusehen. Es ist sehr wahrscheinlich, dass `torch.nn` die Implementierung bereits enthält, sodass du dir das Schreiben eigener Layer sparen kannst. Nachfolgend importieren wir das Paket:

In [34]:
import torch.nn as nn

Zusätzlich zu `torch.nn` gibt es noch `torch.nn.functional`. Dieses Paket enthält Funktionen, die in Netzwerk-Layern verwendet werden. Im Gegensatz dazu definiert `torch.nn` diese Funktionen als `nn.Modules` (mehr dazu weiter unten). `torch.nn` selbst nutzt auch viele Funktionalitäten aus `torch.nn.functional`. Das functional-Paket ist daher in vielen Situationen praktisch – wir importieren es hier ebenfalls.

Module: Kapseln Komponenten eines neuronalen Netzes, während das functional-Paket lose Funktionen bereitstellt.

In [35]:
import torch.nn.functional as F

#### nn.Module

In PyTorch wird ein neuronales Netz aus Modulen aufgebaut. Module können wiederum andere Module enthalten. Das gesamte neuronale Netz wird ebenfalls als Modul betrachtet. Die grundlegende Struktur eines Moduls sieht so aus:

In [36]:
class MyModule(nn.Module):
    def __init__(self): # hier werden alle attribute initialisiert die für das Modul spezifisch sind
        super().__init__() # sorgt dafür, dass ale notwendigen Initialisierungen der Basisklasse nn.Module sichergestellt sind
        # hier könnten jetzt initialisierungen für das modul stehen
        
    def forward(self, x):
        # hier sind die funktionen enthalten, die die berechnungen in dem modul durchführen
        pass

Die `forward`-Funktion definiert die Berechnungen, die innerhalb des Moduls stattfinden. Sie wird ausgeführt, wenn du das Modul aufrufst (z.B. `nn = MyModule(); nn(x)`). In der `__init__`-Funktion erzeugen wir üblicherweise die Parameter des Moduls, indem wir `nn.Parameter` verwenden oder andere Module definieren, die dann in der forward-Funktion zum Einsatz kommen. Die Berechnung der Gradienten (backward) erfolgt automatisch, könnte bei Bedarf aber auch überschrieben werden.

#### Simple Classifier

Nutzen wir nun die vordefinierten Module des `torch.nn`-Pakets, um unser eigenes kleines neuronales Netz zu entwerfen. Wir erstellen ein minimales Netzwerk mit einem Input-Layer, einem versteckten Layer (hidden layer) mit tanh als Aktivierungsfunktion und einem Output-Layer. Unser Netzwerk könnte daher in etwa so aussehen:

<img src="images/small_neural_network.svg" alt="Small neural network" />

Die Eingabeneuronen (blau) repräsentieren die Koordinaten $x1$ und $x2$ eines Datenpunktes. Die Neuronen des versteckten Layers (hidden layer) einschließlich der tanh-Aktivierungsfunktion werden in weiß dargestellt, das Ausgabeneuron in rot. In PyTorch können wir dies folgendermaßen definieren:

In [39]:
class SimpleClassifier(nn.Module):
    
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        # Erstelle Instanzen der Module, die wir für den Aufbau unseres neuronalen Netzes benötigen.
        # Dies umfasst die Definition der Schichten, Aktivierungsfunktionen usw.
        self.linear1 = nn.Linear(num_inputs, num_hidden)
        self.act_fn = nn.Tanh()
        self.linear2 = nn.Linear(num_hidden, num_outputs)
        
    def forward(self, x):
        # Lasse die Eingabe durch das Modell laufen, um dessen Ausgabe (Vorhersage) zu erhalten.
        x = self.linear1(x)
        x = self.act_fn(x)
        x = self.linear2(x)
        return x

Für die Beispiele in diesem Notebook verwenden wir ein kleines neuronales Netz mit zwei Eingabeneuronen und vier Neuronen im versteckten Layer (hidden layer). Da wir eine binäre Klassifikation durchführen, nutzen wir ein einzelnes Ausgabeneuron. Beachte, dass wir noch keine Sigmoid-Funktion auf die Ausgabe anwenden. Dies liegt daran, dass andere Funktionen, insbesondere die Verlustfunktion, effizienter und genauer auf den ursprünglichen Ausgaben (statt den Ausgaben nach der Sigmoid-Funktion) berechnet werden können. Die genauen Gründe dafür besprechen wir später.

In [40]:
model = SimpleClassifier(num_inputs=2, num_hidden=4, num_outputs=1)
# mit print können wir uns alle sub-modules anzeigen lassen
print(model)

SimpleClassifier(
  (linear1): Linear(in_features=2, out_features=4, bias=True)
  (act_fn): Tanh()
  (linear2): Linear(in_features=4, out_features=1, bias=True)
)


Das Ausgeben des Modells auf der Konsole listet alle enthaltenen Untermodule auf. Die Parameter eines Moduls lassen sich mithilfe der Funktionen `parameters()` oder `named_parameters()` abrufen. Letztere versieht zudem jedes Parameterobjekt mit einem Namen. In unserem kleinen neuronalen Netz haben wir die folgenden Parameter:

In [41]:
for name, param in model.named_parameters():
    print(f"Parameter {name}, shape {param.shape}")

Parameter linear1.weight, shape torch.Size([4, 2])
Parameter linear1.bias, shape torch.Size([4])
Parameter linear2.weight, shape torch.Size([1, 4])
Parameter linear2.bias, shape torch.Size([1])


Jedes lineare Layer besitzt eine Gewichtsmatrix mit der Form `[output, input]` sowie einen Bias-Vektor der Form `[output]`. Die Aktivierungsfunktion `tanh` benötigt keine Parameter. Beachte, dass Parameter nur für `nn.Module`-Objekte registriert werden, die direkte Objektattribute sind (also z.B. `self.a = ...`). Wenn du Module innerhalb einer Liste definierst, werden ihre Parameter nicht für das übergeordnete Modul registriert. Dies kann Probleme bei der Optimierung deines Moduls verursachen. Es gibt Alternativen wie `nn.ModuleList`, `nn.ModuleDict` und `nn.Sequential`, die es dir erlauben, verschiedene Datenstrukturen für Module zu verwenden. Wir werden diese Möglichkeiten in späteren Tutorials verwenden und dort näher erläutern.

### Die Daten

### Optimierung

### Training

### Evaluierung

## Zusätzliche Features, die wir noch nicht besprochen haben